## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Tuatapere,NZ,2022-01-27 01:15:20,-46.1333,167.6833,58.82,50,20,9.10,few clouds
1,1,East London,ZA,2022-01-27 01:11:47,-33.0153,27.9116,72.97,92,26,2.84,scattered clouds
2,2,Dicabisagan,PH,2022-01-27 01:15:22,17.0818,122.4157,76.71,91,100,3.20,moderate rain
3,3,Punta Arenas,CL,2022-01-27 01:15:23,-53.1500,-70.9167,50.11,76,40,28.77,scattered clouds
4,4,Lagos,NG,2022-01-27 01:12:04,6.5833,3.7500,71.28,95,0,0.89,clear sky


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 65


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Tuatapere,NZ,2022-01-27 01:15:20,-46.1333,167.6833,58.82,50,20,9.10,few clouds
3,3,Punta Arenas,CL,2022-01-27 01:15:23,-53.1500,-70.9167,50.11,76,40,28.77,scattered clouds
6,6,Ushuaia,AR,2022-01-27 01:15:25,-54.8000,-68.3000,49.66,76,40,27.63,scattered clouds
7,7,San Quintin,MX,2022-01-27 01:15:25,30.4833,-115.9500,59.49,67,100,8.63,overcast clouds
16,16,Naze,JP,2022-01-27 01:15:33,28.3667,129.4833,63.52,69,81,6.58,broken clouds
17,17,Balkanabat,TM,2022-01-27 01:15:34,39.5108,54.3671,49.42,33,100,4.85,overcast clouds
19,19,Brae,GB,2022-01-27 01:15:36,60.3964,-1.3530,46.31,96,100,43.17,overcast clouds
23,23,Ribeira Grande,PT,2022-01-27 01:15:40,38.5167,-28.7000,59.61,78,18,14.65,few clouds
30,30,Guerrero Negro,MX,2022-01-27 01:15:45,27.9769,-114.0611,64.85,47,100,8.63,overcast clouds
32,32,Cameron Park,US,2022-01-27 01:13:13,38.6688,-120.9872,58.93,54,0,1.79,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City_ID            125
City               125
Country            124
Date               125
Lat                125
Lng                125
Max Temp           125
Humidity           125
Cloudiness         125
Wind Speed         125
Current Weather    125
dtype: int64


In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID            124
City               124
Country            124
Date               124
Lat                124
Lng                124
Max Temp           124
Humidity           124
Cloudiness         124
Wind Speed         124
Current Weather    124
dtype: int64

In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Tuatapere,NZ,58.82,few clouds,-46.1333,167.6833,NaN
3,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,NaN
6,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,NaN
7,San Quintin,MX,59.49,overcast clouds,30.4833,-115.9500,NaN
16,Naze,JP,63.52,broken clouds,28.3667,129.4833,NaN
17,Balkanabat,TM,49.42,overcast clouds,39.5108,54.3671,NaN
19,Brae,GB,46.31,overcast clouds,60.3964,-1.3530,NaN
23,Ribeira Grande,PT,59.61,few clouds,38.5167,-28.7000,NaN
30,Guerrero Negro,MX,64.85,overcast clouds,27.9769,-114.0611,NaN
32,Cameron Park,US,58.93,clear sky,38.6688,-120.9872,NaN


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
hotel_df.count()

City               124
Country            124
Max Temp           124
Current Weather    124
Lat                124
Lng                124
Hotel Name         114
dtype: int64

In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
#Reset display options for dataframe just to make sure that all rows are populated
pd.set_option("display.max_rows", None, "display.max_columns", None)
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Tuatapere,NZ,58.82,few clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
3,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
6,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,Albatross Hotel
7,San Quintin,MX,59.49,overcast clouds,30.4833,-115.9500,Don Eddie's Landing
16,Naze,JP,63.52,broken clouds,28.3667,129.4833,Shishime Hotel
17,Balkanabat,TM,49.42,overcast clouds,39.5108,54.3671,Nebitchi Hotel
19,Brae,GB,46.31,overcast clouds,60.3964,-1.3530,Busta House Hotel
23,Ribeira Grande,PT,59.61,few clouds,38.5167,-28.7000,Quinta da Meia Eira
30,Guerrero Negro,MX,64.85,overcast clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
32,Cameron Park,US,58.93,clear sky,38.6688,-120.9872,Quality Inn & Suites Cameron Park Shingle Springs


In [40]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Tuatapere,NZ,58.82,few clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
3,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
6,Ushuaia,AR,49.66,scattered clouds,-54.8000,-68.3000,Albatross Hotel
7,San Quintin,MX,59.49,overcast clouds,30.4833,-115.9500,Don Eddie's Landing
16,Naze,JP,63.52,broken clouds,28.3667,129.4833,Shishime Hotel


In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))